## Modelo de datos de biblioteca de música

Este modelo de datos representa una biblioteca de música que tiene una gran colección de canciones y muestra sus canciones más descargadas casi en tiempo real.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de un disco de canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de descarga específico o todos los registros de descarga de una canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de recuento mensual específico de descargas o todos los registros de recuento de descargas mensuales de una canción, facilitada por una tabla llamada Song.

* Recuperación de todos los registros (incluidos los registros de canciones, los registros de descargas y los registros de recuento de descargas mensuales) de una canción, facilitada por una tabla llamada Songs.

* Buscar la mayoría de las canciones descargadas, facilitado por el índice secundario global de la tabla Canciones llamado DownloadsByMonth.

<br> TABLE:

![image.png](images/song_music01.png)

<br> GSI DownloadsByMonth:

![image.png](images/song_music02.png)

In [1]:
import pandas as pd
from spdynamodb import DynamoTable
from time import sleep
import json
from decimal import Decimal
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

In [2]:
dt = DynamoTable()
try:
    dt.select_table('Songs')
    print(dt)
except:
    dt.create_table(
        table_name='Songs',
        partition_key='Id',
        partition_key_type='S',
        sort_key='Metadata',
        sort_key_type='S',
        provisioned=False
    )

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


- Table name: Songs            
- Table arn: arn:aws:dynamodb:us-east-1:688733876861:table/Songs            
- Table creation: 2023-04-19 11:55:48.633000-03:00            
- [{'AttributeName': 'Id', 'KeyType': 'HASH'}, {'AttributeName': 'Metadata', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Id', 'AttributeType': 'S'}, {'AttributeName': 'Metadata', 'AttributeType': 'S'}]            
- Point-in-time recovery status: ENABLED


In [5]:
dt.load_json("songs.json")

In [ ]:
df = dt.query_items(query="1", to_pandas=True, consumed_capacity=True)
df

* DynamoDB permite la creación de índices secundarios para obtener más patrones de acceso a los datos en la tabla. Los índices secundarios son un medio eficaz para agregar flexibilidad de consulta a la tabla de DynamoDB.

* DynamoDB tiene dos tipos de índices secundarios: los índices secundarios globales y los índices secundarios locales. En esta sección, se agrega un índice secundario global al atributo de categoría.

In [4]:
dt.bill_mode

'PAY_PER_REQUEST'

In [ ]:
dt.create_global_secondary_index(
    att_name="DownloadMonth",
    att_type="S",
    sort_index="TotalDownloadsInMonth",
    sort_type="N",
    i_name="DownloadsByMonth"
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Crear un índice secundario global es muy parecido a crear una tabla. Se especifican el nombre del índice, los atributos que estarán dentro del índice, el esquema de la clave del índice y el rendimiento aprovisionado (la capacidad máxima que una aplicación puede consumir de una tabla o de un índice). El rendimiento aprovisionado en cada índice es independiente del rendimiento aprovisionado en una tabla. Esto permite definir el rendimiento con detalle a fin de satisfacer las necesidades de la aplicación.

In [ ]:
dt.check_status_gsi(index_name='DownloadsByMonth')

### Backup

Cree copias de seguridad de sus tablas para conservarlas a largo plazo y cumplir la normativa. Las acciones de backup y restauración no afectan al desempeño ni a la disponibilidad de la tabla. Cree backups bajo demanda de su tabla o habilite AWS Backup para programar backups automáticamente, incluidos backups de recuperación puntual (PITR).

In [ ]:
# Make a backup of the DynamoDB table
response = dt.make_backup()
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Backup {} created successfully.".format(response['BackupDetails']['BackupName']))

### Recuperación puntual (PITR)
La recuperación puntual ayuda a proteger las tablas de DynamoDB de operaciones de escritura o eliminación accidentales. Con la recuperación puntual, no tiene que preocuparse de crear, mantener o programar copias de seguridad bajo demanda.

Por ejemplo, supongamos que un script de prueba escribe accidentalmente en una tabla de DynamoDB de producción. Con la recuperación puntual, puede restaurar esa tabla en cualquier momento de los últimos 35 días. DynamoDB mantiene copias de seguridad incrementales de la tabla.

In [3]:
dt.status_pitr

'ENABLED'

In [5]:
dt.status_pitr = 'ENABLED'
dt.status_pitr

INFO:spdynamodb:Point-in-time recovery is already ENABLED.


'ENABLED'

In [4]:
# Reading all items from a table
df = dt.read_items(allow_full_scan=True)
df

,Title,Id,TotalDownloads,Metadata,Artist,DownloadTimestamp,DownloadMonth,TotalDownloadsInMonth
0,Paranoid,2,2,Details,Black Sabbath,NaN,NaN,NaN
1,NaN,2,NaN,Dld-9349823684,NaN,2018-01-02T00:00:00,NaN,NaN
2,NaN,2,NaN,Dld-9349823685,NaN,2018-01-03T00:00:02,NaN,NaN
3,NaN,2,NaN,Month-01-2018,NaN,NaN,01-2018,2
4,One,1,3,Details,Metallica,NaN,NaN,NaN
5,NaN,1,NaN,Dld-9349823681,NaN,2018-01-01T00:00:07,NaN,NaN
6,NaN,1,NaN,Dld-9349823682,NaN,2018-01-01T00:20:10,NaN,NaN
7,NaN,1,NaN,Dld-9349823683,NaN,2018-01-02T00:11:10,NaN,NaN
8,NaN,1,NaN,Month-01-2018,NaN,NaN,01-2018,3


In [15]:
# Only reading the columns we need
df = dt.read_items(allow_full_scan=True, columns=["Artist", "Title"])
df.dropna()

,Title,Artist
0,Paranoid,Black Sabbath
4,One,Metallica
